In [6]:
import os
import re
import praw
import time
import string
import pyLDAvis
import numpy as np
import pandas as pd
import seaborn as sns
import pyLDAvis.lda_model
from tabulate import tabulate
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [13]:
reddit = praw.Reddit(
    client_id="OJtDOFbkvxW9oodT0qSozw",
    client_secret="6unjzbEWPaQoomY7uRYq0mQIia80IQ",
    user_agent="moon_rover69"
)

combined_data = []

# subreddit_names = ["UkraineRussiaReport", "RussiaUkraineWar", "RussiaUkraineWar2022", "UkrainianConflict"]
subreddit_names = ["UkrainianConflict"]
for subreddit_name in subreddit_names:
    subreddit = reddit.subreddit(subreddit_name)

    print("Fetching upto 1000 posts and top 50 comments each...")

    for idx, submission in enumerate(subreddit.new(limit=1000)):
        # Add the post itself
        print(f"Fetching comments for post: {idx}: {submission.title}")
        combined_data.append({
            'id': submission.id,
            'type': 'submission',
            'title': submission.title,
            'selftext': submission.selftext,
            'created_utc': submission.created_utc
        })

    # Fetch comments
        submission.comments.replace_more()
        for comment in submission.comments.list()[:50]:
            combined_data.append({
                'id': comment.id,
                'type': 'comment',
                'title': None,
                'selftext': comment.body,
                'created_utc': comment.created_utc
            })

        if (idx + 1) % 100 == 0:
            print(f"{idx + 1} posts processed...")
            time.sleep(1)

df = pd.DataFrame(combined_data)
file_path = 'reddit_combined_posts_and_comments.csv'
# If the file doesn't exist yet, write header; otherwise, append without header
write_header = not os.path.exists(file_path)
df.to_csv(
    "reddit_combined_posts_and_comments.csv",
    mode='a',
    header=write_header,
    index=False
)

print("✅ Done! Data saved with matching format.")

Fetching upto 1000 posts and top 50 comments each...
Fetching comments for post: 0: Germany deploys troops on NATO's eastern flank
Fetching comments for post: 1: Putin Doesn’t Want Peace Because He Claims He’s Winning – But Russian High Command Privately Says His Goals Are "Unachievable"
Fetching comments for post: 2: Russia May Have Deployed Nuclear Air-to-Air Missiles on Fighter Jets. Here’s What US Intelligence Reveals
Fetching comments for post: 3: Trump "thinks Putin is winning the war", his presswoman appears to admitted while refuting WSJ report
Fetching comments for post: 4: Live - Ukraine. Former Putin adviser accidentally reveals the scale of Russian losses in Ukraine during an interview.
Fetching comments for post: 5: Zelenskyy,Mitsotakis discuss pressure on Russia and Trump ties in phone call.There is a lot of diplomatic work happening these days.Strong pressure and additional sanctions against Russia are needed.No one needs to be reminded that the war continues solely beca